# Linear Regression Model

The goal of this analysis is to compare different combinations of variables to determine which subset of predictors creates the **best model** for predicting `lpsa` or `psa`, an antigen which elevanted in the presense of prostate cancer, using the **ProstateData.csv** dataset.

Table of Contenst:
- Setup
- Import Data
- Explore Data Analysis (EDA)
- Build Model
- Evaluate Model
- Model Equation

## Setup

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.feature_selection import RFE

## Import Data

In [2]:
df = pd.read_csv('../data/ProstateData.csv')

## EDA

In [3]:
print('Shape: ', df.shape)
print('Columns: ', df.columns)
print(df.info())
print(df.describe())
df.head()

Shape:  (97, 10)
Columns:  Index(['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45',
       'lpsa', 'train'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   lcavol   97 non-null     float64
 1   lweight  97 non-null     float64
 2   age      97 non-null     int64  
 3   lbph     97 non-null     float64
 4   svi      97 non-null     int64  
 5   lcp      97 non-null     float64
 6   gleason  97 non-null     int64  
 7   pgg45    97 non-null     int64  
 8   lpsa     97 non-null     float64
 9   train    97 non-null     object 
dtypes: float64(5), int64(4), object(1)
memory usage: 7.7+ KB
None
          lcavol    lweight        age       lbph        svi        lcp  \
count  97.000000  97.000000  97.000000  97.000000  97.000000  97.000000   
mean    1.350010   3.628943  63.865979   0.100356   0.216495  -0.179366  

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train
0,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,T
1,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,T
2,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519,T
3,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519,T
4,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564,T


Columns of `df`:
- **lcavol**: log(cancer volume)
- **lweight**: log(prostate weight)
- **age**: age
- **lbph**: log(begining prostate hyperplasia amount)
- **svi**: seminal vesicle invasion. This refers to a tumor infiltration orginating from external tumor cells. There are `0`s and `1`s to represent the 2 different types of **SVI**. 
- **lcp**: log(capsular penetration)
- **gleason**: Gleason score
- **pgg45**: percentage Gleason scores 4 or 5
- **lpsa**: log(prostate specific antigen)
- **train**: logical index used to differentiate train from test points. Consist of `T`s and `F`s.

## Split Data
Since the `df` has already had `train` columns for training (`T`) and testing (`F`) data, no need to manually split `df` into training and testing sets. However, **features** and **target** variables are still needed to be separated into `X` and `y`.


In [4]:
# Create training dataset
trainD = df[df['train'] == 'T'].drop(columns=['train'])
print(f'Number of trained data: {trainD.shape[0]}')
# Create testing dataset
testD = df[df['train'] == 'F'].drop(columns=['train'])
print(f'Number of tested data: {testD.shape[0]}')

Number of trained data: 67
Number of tested data: 30


In [5]:
trainD.head()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
0,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783
1,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519
2,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519
3,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519
4,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564


In [6]:
testD.head()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa
6,0.737164,3.473518,64,0.615186,0,-1.386294,6,0,0.765468
8,-0.776529,3.539509,47,-1.386294,0,-1.386294,6,0,1.047319
9,0.223144,3.244544,63,-1.386294,0,-1.386294,6,0,1.047319
14,1.205971,3.442019,57,-1.386294,0,-0.430783,7,5,1.398717
21,2.059239,3.501043,60,1.474763,0,1.348073,7,20,1.658228


### Define target and predictors

In [7]:
# Define the target (y) and predictors (X)
X = trainD.drop(columns=['lpsa'])  # Predictors
y = trainD['lpsa']  # Target

## Scale Data

In [8]:
# Scale the predictors (important for linear regression)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

## Build Model

In [9]:
model = LinearRegression()

### Recursive Feature Elimination

In [12]:
# Recursive Feature Elimination
rfe = RFE(model, n_features_to_select=7)  # Select 4 features
rfe.fit(trainD.drop(columns=['lpsa']), trainD['lpsa'])

# Selected features
print("Selected features:", trainD.drop(columns=['lpsa']).columns[rfe.support_])

# Fit the best model
best_model = LinearRegression()
best_model.fit(trainD[trainD.drop(columns=['lpsa']).columns[rfe.support_]], trainD['lpsa'])

# Evaluate the model
predictions = best_model.predict(testD[testD.drop(columns=['lpsa']).columns[rfe.support_]])
rmse = root_mean_squared_error(testD['lpsa'], predictions)
r2 = r2_score(testD['lpsa'], predictions)
print("RMSE:", rmse)
print("R-squared:", r2)

Selected features: Index(['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason'], dtype='object')
RMSE: 0.670643930513973
R-squared: 0.5715084468562919


***When `n_features_to_select = 4`***:
``` ruby
Selected features: Index(['lcavol', 'lweight', 'lbph', 'svi'], dtype='object')
RMSE: 0.6755235910315512
R-squared: 0.5652502822042023
```

## Installing mlxtend

In [11]:
@REM conda install mlxtend --channel conda-forge

SyntaxError: invalid syntax (517548891.py, line 1)

### Model Building

#### Forward Selection

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Perform Forward Selection
sfs = SFS(
    model, 
    k_features=7,  # Number of features to select (you can adjust this)
    forward=True,  # Forward selection (set to False for backward selection)
    scoring='r2',  # Metric to evaluate model performance
    cv=5           # Cross-validation folds
)

# Fit the forward selection model
sfs.fit(X, y)

# Display the selected features
print("Selected features:", sfs.k_feature_names_)

Selected features: ('lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'pgg45')


In [16]:
# Extract the selected features
X_selected = X[list(sfs.k_feature_names_)]

# Fit the final model
final_model = LinearRegression()
final_model.fit(X_selected, y)

# Display the coefficients
print("Coefficients:", final_model.coef_)
print("Intercept:", final_model.intercept_)

Coefficients: [ 0.57393039  0.61920883 -0.01947988  0.14442647  0.74178126 -0.20541699
  0.008945  ]
Intercept: 0.2590617470653944


#### Summary Table

##### Version 1

In [17]:
# Perform Forward Selection
sfs2 = SFS(
    model, 
    k_features=(1, X.shape[1]),  # Test all subsets from 1 to all features
    forward=True,  # Forward selection
    scoring='r2',  # Metric to evaluate model performance
    cv=5           # Cross-validation folds
)

# Fit the forward selection model
sfs2.fit(X, y)

# Get the metric dictionary
metric_dict = sfs2.get_metric_dict()

# Convert the metric dictionary to a DataFrame
results_table = pd.DataFrame.from_dict(metric_dict).T

# Debug: Print the metric dictionary
print("Metric Dictionary:")
print(metric_dict)

# Calculate Adjusted R-squared and RSS for each subset
n = len(y)  # Number of observations
results_table['Adjusted R-squared'] = 1 - (1 - results_table['avg_score']) * (n - 1) / (n - results_table['feature_idx'].apply(len) - 1)

# Calculate RSS for each subset
rss_list = []
for feature_idx in results_table['feature_idx']:
    X_subset = X.iloc[:, list(feature_idx)]
    model.fit(X_subset, y)
    y_pred = model.predict(X_subset)
    rss = np.sum((y - y_pred) ** 2)  # RSS = sum of squared residuals
    rss_list.append(rss)

results_table['RSS'] = rss_list

# Display the table
results_table[['feature_names', 'avg_score', 'Adjusted R-squared', 'RSS']]

Metric Dictionary:
{1: {'feature_idx': (0,), 'cv_scores': array([ -3.67997145, -10.99371548, -10.26667086,  -8.77431186,
        -4.15020233]), 'avg_score': np.float64(-7.572974397529393), 'feature_names': ('lcavol',), 'ci_bound': np.float64(3.9519935887741573), 'std_dev': np.float64(3.0747852754478773), 'std_err': np.float64(1.5373926377239386)}, 2: {'feature_idx': (0, 1), 'cv_scores': array([ -2.86506763, -13.106284  ,  -6.35091259,  -6.2267032 ,
        -3.98743112]), 'avg_score': np.float64(-6.507279708487545), 'feature_names': ('lcavol', 'lweight'), 'ci_bound': np.float64(4.571929106786392), 'std_dev': np.float64(3.5571161698920744), 'std_err': np.float64(1.7785580849460372)}, 3: {'feature_idx': (0, 1, 2), 'cv_scores': array([ -2.87749879, -12.98271838,  -6.20363827,  -6.81521966,
        -4.1768807 ]), 'avg_score': np.float64(-6.611191159159733), 'feature_names': ('lcavol', 'lweight', 'age'), 'ci_bound': np.float64(4.477183652389205), 'std_dev': np.float64(3.483400987528519), 'st

,feature_names,avg_score,Adjusted R-squared,RSS
1,"(lcavol,)",-7.572974,-7.704866,44.528583
2,"(lcavol, lweight)",-6.50728,-6.741882,37.091846
3,"(lcavol, lweight, age)",-6.611191,-6.973629,36.817229
4,"(lcavol, lweight, age, lcp)",-6.970353,-7.484569,36.814203
5,"(lcavol, lweight, age, lcp, pgg45)",-7.121519,-7.787218,33.916491
6,"(lcavol, lweight, age, svi, lcp, pgg45)",-7.390986,-8.230085,31.572711
7,"(lcavol, lweight, age, lbph, svi, lcp, pgg45)",-7.641715,-8.667003,29.437300
8,"(lcavol, lweight, age, lbph, svi, lcp, gleason...",-8.137302,-9.397619,29.426384


##### Version 2

In [20]:
sfs2 = SFS(
    model, 
    k_features=(1, X.shape[1]),  # Test all subsets from 1 to all features
    forward=True,  # Forward selection
    scoring='neg_mean_squared_error',  # Use negative MSE instead of R-squared
    cv=3  # Reduce the number of folds for small datasets
)

# Fit the forward selection model
sfs2.fit(X_scaled, y)

# Get the metric dictionary
metric_dict = sfs2.get_metric_dict()
metric_dict

In [22]:
# Convert the metric dictionary to a DataFrame
results_table = pd.DataFrame.from_dict(metric_dict).T

# Calculate Adjusted R-squared and RSS for each subset
n = len(y)  # Number of observations
results_table['Adjusted R-squared'] = 1 - (1 - results_table['avg_score']) * (n - 1) / (n - results_table['feature_idx'].apply(len) - 1)

# Calculate RSS for each subset
rss_list = []
for feature_idx in results_table['feature_idx']:
    X_subset = X_scaled.iloc[:, list(feature_idx)]
    model.fit(X_subset, y)
    y_pred = model.predict(X_subset)
    rss = np.sum((y - y_pred) ** 2)  # RSS = sum of squared residuals
    rss_list.append(rss)

results_table['RSS'] = rss_list

# Display the table
final_table = results_table[['feature_names', 'avg_score', 'Adjusted R-squared', 'RSS']]
final_table

,feature_names,avg_score,Adjusted R-squared,RSS
1,"(lcavol,)",-1.47835,-1.516479,44.528583
2,"(lcavol, lweight)",-1.341568,-1.414742,37.091846
3,"(lcavol, lweight, age)",-1.373316,-1.486331,36.817229
4,"(lcavol, lweight, age, pgg45)",-1.402769,-1.557786,34.815976
5,"(lcavol, lweight, age, lcp, pgg45)",-1.418755,-1.617014,33.916491
6,"(lcavol, lweight, age, lbph, lcp, pgg45)",-1.534291,-1.78772,32.603751
7,"(lcavol, lweight, age, lbph, lcp, gleason, pgg45)",-1.863747,-2.203513,32.519818
8,"(lcavol, lweight, age, lbph, svi, lcp, gleason...",-2.327985,-2.787018,29.426384


In [19]:
# After fitting SFS
selected_features = list(sfs.k_feature_names_)

# After selecting features with SFS/RFE
test_predictions = final_model.predict(testD[selected_features])
test_rmse = root_mean_squared_error(testD['lpsa'], test_predictions)
test_r2 = r2_score(testD['lpsa'], test_predictions)

In [22]:
print(f'Test Predictions: {test_predictions}')
print(f'Test RMSE: {test_rmse}')
print(f'Test R2: {test_r2}')

Test Predictions: [1.95988086 1.17407999 1.25349932 1.90517799 2.5549831  1.92271266
 2.02773043 1.84392403 1.98423025 1.32077751 2.95738085 2.21645891
 2.18741337 2.79439035 2.67390306 2.16798281 2.38839321 3.02295468
 3.2261123  1.40959818 3.42265999 3.70166358 2.53015458 2.74164111
 2.6555575  3.51791852 3.16509553 3.33426708 3.14262448 3.77586284]
Test RMSE: 0.7186887278638796
Test R2: 0.507915217649171


In [21]:
# After fitting RFE
selected_features = trainD.drop(columns=['lpsa']).columns[rfe.support_]
print("Selected features:", selected_features)

# Fit the model on selected features
best_model.fit(trainD[selected_features], trainD['lpsa'])

# Evaluate on the test set
predictions = best_model.predict(testD[selected_features])

Selected features: Index(['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason'], dtype='object')
